In [3]:
import tensorquant as tq
from datetime import date 
import matplotlib.pyplot as plt
import pandas as pd 
# import tensorflow as tf
# import tensorflow_probability as tfp
import numpy as np

# Settings

In [5]:
tq.Settings.evaluation_date = date(2024, 7, 31)
trade_date = tq.Settings.evaluation_date

# Market

In [9]:
market = {}
daycounter = tq.DayCounter(tq.DayCounterConvention.ActualActual)

curves_df = pd.read_excel("data/market_20240731.xlsx", sheet_name='curves')
curves_df['start'] = curves_df['start'].dt.date
curves_df['end'] = curves_df['end'].dt.date
estr_df = curves_df[curves_df['name'] == 'EUR_ESTR']
eur6m_df = curves_df[curves_df['name'] == 'EUR_6M']

rates = estr_df['quote'].values/100
#tq
curve_tq = tq.RateCurve(reference_date=tq.Settings.evaluation_date,
                         pillars=estr_df['end'],
                         rates=rates,
                         interp='LINEAR',
                         daycounter_convention=tq.DayCounterConvention.ActualActual)

vol_df = pd.read_excel("data/market_20240731.xlsx", sheet_name = 'vols', index_col=0)
default_vol = vol_df.loc['DEFAULT']
maturities = default_vol['tenor'].unique()
strikes = default_vol['strike'].unique()

vol_matr = np.zeros((len(maturities), len(strikes)))
for i, t in enumerate(maturities):
    vol_matr[i, :] = default_vol[default_vol['tenor'] == t]['quote'].values
    
surface = tq.VolatilitySurface(reference_date=tq.Settings.evaluation_date,
                            calendar=tq.TARGET(),
                            daycounter=tq.DayCounter(tq.DayCounterConvention.Actual365),
                            strike=strikes,
                            maturity=maturities/365,
                            volatility_matrix=vol_matr)

spot_df = pd.read_excel("data/market_20240731.xlsx", sheet_name = 'spot', index_col=0)

market['IR:EUR:ESTR'] = curve_tq
market['VOLEQ:DEFAULT'] = surface
market['EQ:DEFAULT'] = spot_df.loc['DEFAULT']['quote']

# Option

# 1. Black-Scholes Model Overview

The **Black-Scholes model** is a mathematical model for pricing European-style options. The model assumes that the price of the underlying asset follows a geometric Brownian motion with constant drift and volatility.

The Black-Scholes formula for the price of a European call option is given by:

$$
C(S, t) = S_0 N(d_1) - X e^{-r(T-t)} N(d_2)
$$

where:
- $ S_0 $ is the current price of the asset.
- $ X $ is the strike price.
- $ T $ is the time to expiration.
- $ r $ is the risk-free interest rate.
- $ N(d) $ is the cumulative distribution function of the standard normal distribution.
- $ d_1 $ and $ d_2 $ are given by:

$$
d_1 = \frac{\ln(S_0/X) + (r + \frac{\sigma^2}{2}) (T-t)}{\sigma \sqrt{T-t}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T-t}
$$

### Assumptions of the Model:
1. The price of the underlying asset follows a lognormal distribution.
2. There are no transaction costs or taxes.
3. The risk-free interest rate is constant.
4. The volatility of the asset is constant.
5. The options are European-style, meaning they can only be exercised at expiration.

---

# 2. Key Sensitivities (Greeks)

In the Black-Scholes framework, the price of an option is sensitive to various factors, commonly referred to as the **Greeks**. The most important Greeks are:

- **Delta (Δ)**: Measures the sensitivity of the option price to small changes in the price of the underlying asset. Mathematically, it is the partial derivative of the option price with respect to the asset price.

  $$
  \Delta = \frac{\partial C}{\partial S_0}
  $$

- **Gamma (Γ)**: Measures the sensitivity of Delta to small changes in the price of the underlying asset.

  $$
  \Gamma = \frac{\partial^2 C}{\partial S_0^2}
  $$

- **Theta (Θ)**: Measures the sensitivity of the option price to the passage of time. It is the partial derivative of the option price with respect to time.

  $$
  \Theta = \frac{\partial C}{\partial t}
  $$

- **Vega (ν)**: Measures the sensitivity of the option price to changes in volatility of the underlying asset.

  $$
  \nu = \frac{\partial C}{\partial \sigma}
  $$

- **Rho (ρ)**: Measures the sensitivity of the option price to changes in the risk-free interest rate.

  $$
  \rho = \frac{\partial C}{\partial r}
  $$


In [10]:
maturity_date = tq.TARGET().advance(tq.Settings.evaluation_date, 6, tq.TimeUnit.Months, tq.BusinessDayConvention.ModifiedFollowing)
strike_price = 130

option = tq.VanillaOption(tq.Currency.EUR, tq.Settings.evaluation_date, maturity_date, tq.OptionType.Call,
                             underlying = 'DEFAULT',strike = strike_price)
black_pricer = tq.BlackScholesPricer(tq.market_map)
black_pricer.price(option, market, True)

In [11]:
option.price

<tf.Tensor: shape=(), dtype=float32, numpy=11.509682>

In [12]:
black_pricer.tape.gradient(option.price, [black_pricer._s, black_pricer._r, black_pricer._sigma, black_pricer._t])


[<tf.Tensor: shape=(), dtype=float32, numpy=0.5430066>,
 <tf.Tensor: shape=(), dtype=float32, numpy=29.131895>,
 <tf.Tensor: shape=(), dtype=float32, numpy=35.938335>,
 <tf.Tensor: shape=(), dtype=float32, numpy=13.412939>]

# 3. Monte Carlo Simulation for Option Pricing

Monte Carlo simulations are used to model the behavior of financial instruments under uncertainty. In the context of option pricing, Monte Carlo methods involve simulating numerous paths for the price of the underlying asset using random samples from a probability distribution (typically normal distribution).

### Steps in a Monte Carlo Simulation for Option Pricing:
1. Simulate multiple paths for the underlying asset price using a stochastic process like **Geometric Brownian Motion (GBM)**:
   
   $$
   S_t = S_0 e^{(r - \frac{\sigma^2}{2})t + \sigma W_t}
   $$
   
   where $ W_t $ is a Wiener process (Brownian motion).

2. Calculate the payoff for each simulated path at option maturity. For a call option, the payoff is:
   
   $$
   \max(S_T - X, 0)
   $$
   
3. Discount the average payoff back to the present to get the option price:
   
   $$
   C = e^{-rT} \frac{1}{n} \sum_{i=1}^{n} \max(S_T^i - X, 0)
   $$

By simulating a large number of paths, the Monte Carlo approach provides an approximation for the option price, particularly useful for options with complex features that make closed-form solutions difficult.

In [13]:
s0 = black_pricer._s.numpy()
r = black_pricer._r.numpy()
sigma = black_pricer._sigma.numpy()
t = black_pricer._t.numpy()
gbm = tq.GeometricBrownianMotion(r,sigma, s0)

In [14]:
import tensorflow as tf
n_path = 200000
timesteps = 200
z = tf.random.normal((n_path, timesteps), seed=12, dtype=tf.dtypes.float64)

In [15]:
t = tf.Variable(t, dtype=tf.float64)
with tf.GradientTape() as tape:
    s_t = gbm.evolve(t, z)
    payoff = tf.math.maximum(s_t[:, -1] - strike_price, 0)
    p_mc = tf.exp(-gbm._mu * t) * tf.reduce_mean(payoff)

sensy = tape.gradient(p_mc, [gbm._x0, gbm._mu, gbm._sigma, t])
print("Black AAD: ")
print("Price: ", p_mc.numpy())
print("*"*30)
print(f"Delta: {sensy[0].numpy()}")
print(f"Rho: {sensy[1].numpy()}")
print(f"Vega: {sensy[2].numpy()}")
print(f"Theta: {sensy[3].numpy()}")
print("*"*30)

Black AAD: 
Price:  11.549910803860488
******************************
Delta: 0.542190725790068
Rho: 29.059129051217027
Vega: 36.12936387712836
Theta: 13.46892131636077
******************************
